### Tarea 3, Métodos Numéricos, Miguel Angel Ruiz Ortiz

In [29]:
import numpy as np
import pandas as pd
from typing import Tuple, Optional, List
from numerical_methods import solveByGaussianElimination

### Problema 1

-> Aplica el método de la potencia y potencia inversa para el cálculo del autovalor y autovector dominante de la matriz propuesta $A$ y su inversa. Da la tabla de iterados, la tolerancia usada y el vector inicial.
$$ A = \begin{pmatrix} 
15 & -2 & 2 \\
1 & 10 & -3 \\
-2 & 1 & 0
\end{pmatrix}

In [30]:
def MetodoPotenciaRayleigh(
        A: np.array, 
        x0: np.array, 
        TOL: float = 10**(-5), 
        MAX_ITER: int = 1000,
        iterations: Optional[List[float]] = None
        ) -> Tuple[float, np.array]:
    """Método de la Potencia utilizando el coeficiente de Rayleigh (x^TAx, cuando ||x||=1) para calcular el eigenvalor de mayor magnitud y su eigenvector 
    asociado de una matriz A de nxn.

    :param A: Matriz de nxn a la cual se le quiere calcular el eigenvalor de mayor magnitud y su eigenvector asociado.
    :type A: np.array
    :param x0: Vector inicial de tamaño n para el algoritmo iterativo
    :type x0: np.array
    :param TOL: Tolerancia para saber cuando convergió el algoritmo, i.e., cuando ||Ax-lambda*x|| < TOL donde x es el eigenvector y lambda el eigenvalor, defaults to 10**(-5)
    :type TOL: float, optional
    :param MAX_ITER: Número máximo de iteraciones del algoritmo, defaults to 1000
    :type MAX_ITER: int, optional
    :param iterations: Lista vacía opcional para guardar "inplace" las iteraciones de las lambdas, defaults to None
    :type iterations: Optional[List[float]], optional
    :raises Exception: Si no converge el algoritmo, i.e., se pasa del número de iteraciones máximo, se levanta una excepción
    :return: Tupla de eigenvalor de mayor magnitud y su eigenvector asociado normalizado
    :rtype: Tuple[float, np.array]
    """ 
    A = A.astype("float64")
    xk = x0/np.linalg.norm(x0)
    
    lambda_k = np.inf
    
    for iter in range(MAX_ITER):
        yk = A@xk
        xk = yk / np.linalg.norm(yk)
        
        lambda_k = xk.T@A@xk # cociente de Rayleigh
        
        if iterations is not None:
            iterations.append(lambda_k)
        
        if np.linalg.norm(A@xk - lambda_k*xk) < TOL:
            print(f"Converge en {iter+1} iteraciones")
            return lambda_k, xk
        
    raise Exception("No converge")

def MetodoPotenciaInversaDesplazamiento(
    A: np.array, 
    x0: np.array, 
    p: float = 0, 
    TOL: float = 10**(-5), 
    MAX_ITER: int = 1000,
    iterations: Optional[List[float]] = None
    ) -> Tuple[float, np.array]:
    """Método de la Potencia Inversa con desplazamiento para calcular el eigenvalor de una matriz A más cercano a un valor dado p. p por default es 0, 
    por lo que si no se especifica el algoritmo regresa el eigenvalor de A más pequeño en magnitud (Método de la Potencia Inversa normal).

    :param A:  Matriz de nxn a la cual se le quiere calcular el eigenvalor más cercano a p (si p no se especifica, el de menor magnitud) y su eigenvector asociado.
    :type A: np.array
    :param x0: Vector inicial de tamaño n para el algoritmo iterativo
    :type x0: np.array
    :param p: Target para encontrat el eigenvalor más cercano a p, si p no es dado se obtiene el eigenvalor más pequeño en magnitud, defaults to 0
    :type p: float, optional
    :param TOL: Tolerancia para saber cuando convergió el algoritmo, i.e., cuando ||Ax-lambda*x|| < TOL donde x es el eigenvector y lambda el eigenvalor, defaults to 10**(-5)
    :type TOL: float, optional
    :param MAX_ITER: Número máximo de iteraciones del algoritmo, defaults to 1000
    :type MAX_ITER: int, optional
    :param iterations: Lista vacía opcional para guardar "inplace" las iteraciones de las lambdas, defaults to None
    :type iterations: Optional[List[float]], optional
    :raises Exception: Si no converge el algoritmo, i.e., se pasa del número de iteraciones máximo, se levanta una excepción
    :return: Tupla de eigenvalor más cercano a p (o eigenvalor más pequeño en magnitud si p no es dado) y su eigenvector asociado normalizado
    :rtype: Tuple[float, np.array]
    """    
    A = A.astype("float64")
    xk = x0/np.linalg.norm(x0)
    n = x0.size
    A_shift = A-p*np.identity(n)
    
    lambda_k = np.inf
    
    for iter in range(MAX_ITER):
        yk = solveByGaussianElimination(A_shift, xk) # resolvemos A@yk = xk
        xk = yk / np.linalg.norm(yk)
        
        lambda_k = xk.T@A_shift@xk # cociente de Rayleigh
        
        if iterations is not None:
            iterations.append(lambda_k + p)
        
        if np.linalg.norm(A_shift@xk - lambda_k*xk) < TOL:
            print(f"Converge en {iter+1} iteraciones")
            return lambda_k+p, xk
        
    raise Exception("No converge")

In [31]:
A = np.array([
    [15, -2, 2],
    [1, 10, -3],
    [-2, 1, 0]
    ], dtype="float64")

Se utilizó la misma tolerancia, la cual fue $10^{-5}$, y el mismo vector inicial para los diferentes puntos de este ejercicio. El vector inicial es el siguiente:

In [32]:
np.random.seed(0)
x0 = np.random.random(A.shape[0])
x0 # vector inicial

array([0.5488135 , 0.71518937, 0.60276338])

Cálculo del eigenvalor de mayor magnitud y su eigenvector asociado, y debajo su tabla de iterados:

In [33]:
lambdas_iter = [] # lista en donde se guardarán las iteraciones de lambda^(k)
lambda1, v1 = MetodoPotenciaRayleigh(A, x0, iterations=lambdas_iter)
lambda1, v1

Converge en 41 iteraciones


(14.102548953293946, array([ 0.94359156,  0.31169603, -0.11171641]))

In [34]:
pd.DataFrame({"lambda^(k)": lambdas_iter}, index=[f"Iteración {iter}" for iter in range(1, len(lambdas_iter)+1)]) # Tabla de iterados

,lambda^(k)
Iteración 1,12.792862
Iteración 2,13.120075
Iteración 3,13.355859
Iteración 4,13.542714
Iteración 5,13.686626
Iteración 6,13.795206
Iteración 7,13.876098
Iteración 8,13.935930
Iteración 9,13.980017
Iteración 10,14.012445


Cálculo del eigenvalor de menor magnitud y su eigenvector asociado, y debajo su tabla de iterados:

In [35]:
lambdas_iter_2 = [] # lista en donde se guardarán las iteraciones de lambda^(k)
lambda3, v3 = MetodoPotenciaInversaDesplazamiento(A, x0, iterations=lambdas_iter_2)
lambda3, v3

Converge en 5 iteraciones


(0.5120851859957193, array([-0.08811715,  0.30873883,  0.94705634]))

In [36]:
pd.DataFrame({"lambda^(k)": lambdas_iter_2}, index=[f"Iteración {iter}" for iter in range(1, len(lambdas_iter_2)+1)]) # Tabla de iterados

,lambda^(k)
Iteración 1,0.576570
Iteración 2,0.514433
Iteración 3,0.512214
Iteración 4,0.512092
Iteración 5,0.512085


Corroboración con librería de Numpy:

In [37]:
np.linalg.eig(A)

(array([ 0.51208483, 14.10255576, 10.38535941]),
 array([[-0.08811726, -0.94359219,  0.39292879],
        [ 0.30873868, -0.31169403,  0.91947889],
        [ 0.94705637,  0.11171665,  0.01286632]]))

-> a) Con los resultados obtenidos da una aproximación para el número de condición de la matriz.

Una estimación para el número de condición de A es 
$$\frac{|\lambda_{max}|}{|\lambda_{min}|},$$
donde $\lambda_{max}$ y $\lambda_{min}$ son los eigenvalores de A de mayor y menor magnitud, respectivamente (explicación en PDF).

In [38]:
condition_number_2 = np.linalg.norm(A, ord=2)*np.linalg.norm(np.linalg.inv(A), ord=2) # número de condición de A usando norma 2
abs(lambda1)/abs(lambda3), condition_number_2 # si lambda1 es el eigenvalor de mayor magnitud y lambda3 es el de menor, entonces |lamda1|/|lambda3| estima por debajo al número de condición de A

(27.539458939575404, 33.24309351411975)

-> b) Implementa a alguna técnica para disminuir el número de iterados obtenidos. Muestra la nueva tabla de iterados.

Se obtiene el eigenvalor de mayor magnitud de A con su eigenvector asociado en la mitad de iteraciones (para el mismo vector inicial), y debajo se muestra la tabla de iterados. Explicación en PDF.

In [39]:
lambdas_iter_3 = []
lambda1, v1 = MetodoPotenciaInversaDesplazamiento(A, x0, p=19, iterations=lambdas_iter_3)
lambda1, v1

Converge en 22 iteraciones


(14.10254808678905, array([ 0.94359148,  0.31169628, -0.11171638]))

In [40]:
pd.DataFrame({"lambda^(k)": lambdas_iter_3}, index=[f"Iteración {iter}" for iter in range(1, len(lambdas_iter_3)+1)]) 

,lambda^(k)
Iteración 1,12.035112
Iteración 2,13.404617
Iteración 3,13.753713
Iteración 4,13.906008
Iteración 5,13.990099
Iteración 6,14.038323
Iteración 7,14.065956
Iteración 8,14.081729
Iteración 9,14.090712
Iteración 10,14.095822


### Problema 4

-> Usa una técnica numérica para determinar todos los valores propios y los vectores propios asociados a la matriz:
$$
\begin{pmatrix}
4 & 2 & 0 & 0 & 0 \\
2 & 4 & 2 & 0 & 0 \\
0 & 2 & 4 & 2 & 0 \\
0 & 0 & 2 & 4 & 2 \\
0 & 0 & 0 & 2 & 4
\end{pmatrix}
$$
¿Qué técnica usaste? Da muestra de tus resultados computacionales.

Explicación en PDF.

In [41]:
def HouseholderMatrix(v: np.array) -> np.array:
    return np.identity(v.size) - (2/(np.linalg.norm(v)**2))*np.outer(v,v.T)

def canonical_vector(size: int, index: int) -> np.array:
    e = np.zeros(size)
    e[index] = 1.0
    return e

def MetodoPotenciaConHouseholder(A: np.array, TOL: float = 10**(-5), MAX_ITER: int = 1000) -> np.array:
    n = A.shape[0]
    
    valores_propios = np.zeros(n)
    B = A.copy()
    
    for i in range(n):
        if i == n-1:
            # ultima iteración, B sólo tiene el último valor propio
            valores_propios[i] = B[0,0]
            continue
        
        valores_propios[i], vk = MetodoPotenciaRayleigh(B, np.random.random(B.shape[0]), TOL=TOL, MAX_ITER=MAX_ITER) 
        e1 = canonical_vector(vk.size, 0)
        v = vk - e1
        H = HouseholderMatrix(v)
        B1 = H@B@H
        B = B1[1:, 1:]
    
    return valores_propios

In [42]:
A = np.array([
    [4, 2, 0, 0, 0],
    [2, 4, 2, 0, 0],
    [0, 2, 4, 2, 0],
    [0, 0, 2, 4, 2],
    [0, 0, 0, 2, 4]
], dtype="float64")

In [43]:
lambdas = MetodoPotenciaConHouseholder(A)
lambdas # eigenvalores de A

Converge en 46 iteraciones
Converge en 32 iteraciones
Converge en 19 iteraciones
Converge en 11 iteraciones


array([7.46410162, 6.        , 4.        , 2.        , 0.53589838])

In [44]:
eigenvectores = []
epsilon = 10**(-3)
for k in range(lambdas.size):
    lambdak, vk = MetodoPotenciaInversaDesplazamiento(A, np.random.random(A.shape[0]), p=lambdas[k]+epsilon)
    eigenvectores.append(vk)

Converge en 2 iteraciones
Converge en 2 iteraciones
Converge en 2 iteraciones
Converge en 2 iteraciones
Converge en 2 iteraciones


Los eigenvalores y eigenvectores son los siguientes:

In [45]:
print("Eigenvalores:", lambdas)
print("Eigenvectores:", eigenvectores)

Eigenvalores: [7.46410162 6.         4.         2.         0.53589838]
Eigenvectores: [array([0.28867514, 0.49999999, 0.57735026, 0.50000001, 0.28867516]), array([ 5.00000729e-01,  5.00000993e-01,  9.04749352e-07, -4.99998983e-01,
       -4.99999295e-01]), array([ 5.77350345e-01,  8.63012346e-08, -5.77350175e-01,  8.22457549e-10,
        5.77350288e-01]), array([-4.99998944e-01,  5.00001421e-01, -2.68946020e-06, -4.99998459e-01,
        5.00001176e-01]), array([ 0.28867465, -0.49999854,  0.5773504 , -0.50000085,  0.28867642])]


Corroboración con librería de Numpy

In [46]:
np.linalg.eig(A)

(array([7.46410162, 6.        , 4.        , 0.53589838, 2.        ]),
 array([[ 2.88675135e-01, -5.00000000e-01,  5.77350269e-01,
         -2.88675135e-01, -5.00000000e-01],
        [ 5.00000000e-01, -5.00000000e-01,  4.16874298e-16,
          5.00000000e-01,  5.00000000e-01],
        [ 5.77350269e-01, -9.38463736e-16, -5.77350269e-01,
         -5.77350269e-01,  8.73792257e-17],
        [ 5.00000000e-01,  5.00000000e-01, -1.12283347e-16,
          5.00000000e-01, -5.00000000e-01],
        [ 2.88675135e-01,  5.00000000e-01,  5.77350269e-01,
         -2.88675135e-01,  5.00000000e-01]]))